# Recuperação da informação para conjunto de e-mails

Foram utilizadas as bases `20_newsgroups.tar.gz` e `mini_newsgroups.tar.gz` para o projeto:

[ICS: E-mail groups](https://kdd.ics.uci.edu/databases/20newsgroups/20newsgroups.html)

<div style="padding: 20px; 
background-color: #de841d; 
color: white; 
width: 80vw;
border-radius: 8px;">
  <h2>Atenção</h2>
  <p>A base de dados possui um total de 22.000 elementos. Destes, 2.000 são utilizados como query, e os outros 20.000 são para consulta. 
    <br><br> 
    Após a tokenização, o vocabulário pode chegar a 130.000 palavras, resultando em uma matriz com o termo para cada documento, o que pode ocupar até 30GB de RAM.
    <br><br>
    Portanto, é importante ter cuidado ao executar o processo, pois é necessário um grande volume de RAM para sua execução.</div>


In [1]:
# Importa os módulos necessários
import os    # Módulo para lidar com funções do sistema operacional
import gc    # Módulo para realizar coleta de lixo e gerenciamento de memória
import sys

import numpy as np   # Módulo para trabalhar com matrizes e funções matemáticas
import pandas as pd  # Módulo para trabalhar com dataframes e séries em Python

from ir.preprocessing import lemmatize_word  # Importa a função de lematização de palavras
from ir.tf_idf import tfidf  # Importa a função de cálculo de TF-IDF

from sklearn.metrics.pairwise import linear_kernel

<div></div> 

## Leitura dos Arquivos

Para a pasta da base de dados, os arquivos foram fornecidos em um formato raw, sem indicação de extensão. Cada e-mail é um arquivo dentro de uma pasta que representa um tema.

Portanto, nesse caso, é necessário percorrer cada pasta para realizar a leitura e armazenamento da base de dados para análises futuras.

<div></div> 

In [2]:
# caminho das queries 
query_path = '../data/emails/mini_newsgroups/'

# caminho dos documentos
docs_path = '../data/emails/20_newsgroups/'

# Iterate over each file in the directory and its subdirectories
def process_files(doc_dir: str): 
    
    database = [] 
    
    for filepath in os.listdir(doc_dir): 
        
        for filename in os.listdir(f'{doc_dir}{filepath}'):

            # Open each file individually and read its contents
            with open(os.path.join(doc_dir, filepath, filename), 'r') as f:
                text_data = f.read().strip()

            # Split the header and body of the email
            try:
                header, body = text_data.split('\n\n', maxsplit=1)
            except:
                continue

            # Convert header to a dictionary
            # header_dict = {}
            # for line in header.split('\n'):
            #     try:
            #         # Split the key and value in each header field and store them in a dictionary
            #         key, value = line.strip().split(': ', maxsplit=1)
            #         header_dict[key] = value
            #     except:
            #         # If a header field cannot be split properly, skip it and continue
            #         continue

            # Append the processed data to the list

            database.append({'filepath': filepath, 
                            'filename': filename,
                            'body': body, 
                            # **header_dict,
                            # 'text': text_data
                            })
    return database

# tranformation from dict -> dataframe
base_doc = pd.DataFrame(process_files(docs_path))

base_que = pd.DataFrame(process_files(query_path))

# Marcação das bases
base_doc['tag'] = 'doc'
base_que['tag'] = 'query'

# Amostragem para testes
base_doc = base_doc.sample(frac=0.5, random_state=42)

# junção das bases 
base = pd.concat([base_doc, base_que])
base.reset_index(drop=True, inplace=True)

del base_doc, base_que

# remove database from memory
gc.collect()

0

<div></div> 

## Pré-Processamento de Texto

Para minimizar possíveis gargalos de processamento e identificação dos termos relevantes, é realizada a remoção de ruídos utilizando regex. Em seguida, é aplicada a tokenização, que consiste na transformação do texto em uma lista de palavras, a fim de possibilitar a aplicação das técnicas de TF-IDF em um modelo vetorial.

<div></div> 

### Remoção de palavras e transformação de minúsculos


In [3]:
# (\[a-z]): para encontrar todos os caracteres que começam com uma barra invertida () seguida por uma letra minúscula (a-z);
# ([^\w\]): para encontrar todos os caracteres que não são letras, números ou barras invertidas ();
# (\S+\d\S+): para encontrar todos os trechos de texto que contêm um ou mais caracteres não brancos (\S), 
# seguidos por um dígito (\d), seguidos por mais um ou mais caracteres não brancos (\S).
base['post'] = base['body'].replace(r'(\\[a-z])|([^\w\\])|(\S+\d\S+)', ' ', regex=True)

# Aplicando as funções str.lower() e str.strip() simultaneamente
base['post'] = base['post'].apply(lambda x: x.lower().strip())


<div></div> 

### Tokenização e Lemmatizer

**Tokenização:** A tokenização de texto é o processo de dividir um texto em unidades menores, chamadas de tokens. Esses tokens podem ser palavras individuais, caracteres, frases ou até mesmo partes específicas de um texto, dependendo do contexto e das necessidades do processamento de linguagem natural. 

**Lemmatize:** A lematização de texto é um processo linguístico que visa reduzir as palavras em sua forma base ou forma lematizada. O objetivo é transformar palavras flexionadas em sua forma canônica, chamada de "lema" ou "base". Por exemplo, a lematização transforma palavras como "correndo" em "correr", "carros" em "carro" e assim por diante.

In [4]:
base['post'] = base['post'].apply(lambda x: ' '.join([lemmatize_word(word.lower()) for word in x.split()]))

0        todd steve write chuck petch write now it appe...
1        boston globe wednesday april 21 col 4 bodie fo...
2        nl chicago wait til next year new york bunch o...
3        i recent had a case of shingle and my doctor w...
4        for sale 2 amiga commodore amiga best offer ra...
                               ...                        
11972    in article huston acces digex com herb huston ...
11973    i just start read thi newsgroup and haven t be...
11974    blesed are those who hunger and thirst for rig...
11975    i m curiou to know if christia ever read book ...
11976    in article prl csi dit csiro au peter lamb wri...
Name: post, Length: 11977, dtype: object

### Identificação das query / docs

Foi feita uma separação do index das query, para pode fazer uma localização do na base origina após o TF-IDF, dado que o TF-IDF reseta os index dos termos por documento

In [36]:
d_index = base.query('tag=="doc"').index
q_index = base.query('tag=="query"').index

## Processamento dos dados

Aplicação das técnicas estatísticas no conjunto de palavras por documento

### TF IDF

TF-IDF (Term Frequency-Inverse Document Frequency) é uma medida estatística usada para avaliar a importância de um termo em um documento em relação a uma coleção de documentos. É amplamente utilizado em processamento de linguagem natural e recuperação de informações.

O TF-IDF é calculado levando em consideração dois fatores principais:

Frequência do termo (TF - Term Frequency): Mede a frequência com que um termo específico aparece em um documento. Quanto mais vezes um termo aparece, maior é sua relevância no documento.

Frequência inversa do documento (IDF - Inverse Document Frequency): Mede a raridade de um termo em relação a uma coleção de documentos. Quanto menos frequente um termo é em outros documentos da coleção, maior é o seu valor IDF e maior será seu peso para distinguir a importância desse termo no documento atual.

O TF-IDF é calculado multiplicando-se o TF pelo IDF para cada termo em um documento. Dessa forma, termos frequentes no documento e raros na coleção terão um valor TF-IDF mais alto, indicando sua relevância para o documento em questão.

Essa medida é amplamente utilizada em tarefas como recuperação de informações, classificação de texto, sumarização automática e agrupamento de documentos.

In [ ]:
weights = tfidf(base, 'post').T

## Ranqueamento

O ranqueamento de documentos utilizando o TF-IDF (Term Frequency-Inverse Document Frequency) é um método utilizado para ordenar documentos em uma coleção com base na relevância em relação a uma consulta de busca.

Nesse método, cada documento é representado por um vetor numérico, no qual cada dimensão corresponde a um termo presente na coleção de documentos. O valor de cada dimensão é calculado utilizando a fórmula do TF-IDF, que leva em consideração a frequência do termo no documento e a raridade do termo na coleção.

In [ ]:
rank_geral = linear_kernel(weights.iloc[d_index], weights.iloc[q_index])
rank_geral = pd.DataFrame(rank_geral, index=d_index, columns=q_index)

In [ ]:
def calcular_resultados_relevantes(q_index: list, base: pd.DataFrame) -> 'resultados_relevantes[dict], resultados_sistema[dict]':
    resultados_sistema = {}

    for q in q_index: 
        resultados_sistema[q] = rank_geral[q].sort_values(ascending=False).index

    resultados_relevantes = {}

    for q in q_index:
        q_genre = base.iloc[q]['genres']

        k = []

        for d in resultados_sistema[q]:
            d_genre = base.iloc[d]['genres']
            
            # Verifica qual lista de gêneros é menor para otimizar a comparação
            if len(d_genre) > len(q_genre):
                comparativo_menor = q_genre
                comparativo_maior = d_genre
            else:
                comparativo_menor = d_genre
                comparativo_maior = q_genre
            
            # Verifica se há pelo menos um gênero em comum entre as listas
            partial_relevance = any(i in comparativo_maior for i in comparativo_menor)
            
            if partial_relevance:
                k.append(d)
        
        print(f'\rQuery: {q}/{q_index.max()} - Doc: {d}/{d_index.max()}', end='')
        sys.stdout.flush()

        resultados_relevantes[q] = k
        
    return resultados_relevantes, resultados_sistema

resultados_relevantes, resultados_sistema = calcular_resultados_relevantes(q_index, base)


Query: 3093/3093 - Doc: 1730/3092

         3 function calls in 0.000 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.000    0.000 {built-in method builtins.exec}
        1    0.000    0.000    0.000    0.000 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}

## Métricas

P@N: essa métrica mede a proporção de documentos relevantes presentes entre os 10 primeiros resultados retornados por um sistema de busca em resposta a uma consulta.

MAP (Mean Average Precision): o MAP leva em consideração a precisão e a ordenação dos resultados retornados por um sistema de busca em relação a um conjunto de consultas. Ele mede a média das precisões médias de cada consulta.

In [1]:
def calcular_p_n_media(resultados_relevantes, resultados_sistema, n):
    """
    Calcula a média da precisão P@n para um conjunto de consultas e seus resultados relevantes.

    Parâmetros:
    - resultados_relevantes (dict): Um dicionário que mapeia cada consulta aos seus resultados relevantes.
    - resultados_sistema (dict): Um dicionário que mapeia cada consulta aos resultados retornados pelo sistema.
    - n (int): O número de resultados a considerar para o cálculo da precisão.

    Retorno:
    - p_n_media (float): A média da precisão P@n para todas as consultas.

    """
    def calcular_p_n(resultados, relevantes):
        """
        Calcula a precisão P@n para uma lista de resultados e seus resultados relevantes.

        Parâmetros:
        - resultados (list): Uma lista de resultados retornados pelo sistema.
        - relevantes (list): Uma lista de resultados relevantes para a consulta.

        Retorno:
        - p_n (float): A precisão P@n.

        """
        if len(resultados) > n:
            resultados = resultados[:n]  # Considerar apenas os primeiros n resultados
        num_relevantes = len(set(resultados) & set(relevantes))  # Contar quantos resultados relevantes foram encontrados
        p_n = num_relevantes / n  # Calcular a precisão P@n
        return p_n

    p_n_total = 0
    for consulta, relevantes in resultados_relevantes.items():
        resultados = resultados_sistema.get(consulta, [])  # Obtém os resultados retornados pelo sistema para a consulta
        p_n = calcular_p_n(resultados, relevantes)
        p_n_total += p_n

    p_n_media = p_n_total / len(resultados_relevantes)
    return p_n_media


In [ ]:
for x in [10, 20, 50, 100]: 
    print(f"Média do P@{x}: {calcular_p_n_media(resultados_relevantes, resultados_sistema, n=x)}")

Média do P@10: 0.719268030139934
Média do P@20: 0.6967168998923577
Média do P@50: 0.6745748116254034
Média do P@100: 0.658902045209902


In [ ]:
def average_precision(relevantes, recomendados):
    """
    Calcula a Média de Precisão (Average Precision) para um conjunto de itens relevantes e itens recomendados.

    Parâmetros:
    - relevantes (list): Uma lista contendo os itens relevantes.
    - recomendados (list): Uma lista contendo os itens recomendados.

    Retorno:
    - ap (float): O valor da Média de Precisão.

    """
    relevancia_cumulativa = 0
    precision_cumulativa = 0
    num_relevantes = len(relevantes)
    ap = 0

    for i, rec in enumerate(recomendados):
        if rec in relevantes:
            relevancia_cumulativa += 1
            precision_cumulativa += relevancia_cumulativa / (i + 1)

    if num_relevantes > 0:
        ap = precision_cumulativa / num_relevantes

    return ap


def mean_average_precision(resultados_relevantes, resultados_sistema):
    """
    Calcula a Média de Precisão (MAP) para um conjunto de consultas, seus resultados relevantes e resultados retornados pelo sistema.

    Parâmetros:
    - resultados_relevantes (dict): Um dicionário que mapeia cada consulta aos seus resultados relevantes.
    - resultados_sistema (dict): Um dicionário que mapeia cada consulta aos resultados retornados pelo sistema.

    Retorno:
    - map (float): O valor da Média de Precisão Média (MAP) para todas as consultas.

    """
    map = 0
    num_consultas = len(resultados_relevantes)

    for q in resultados_relevantes:
        relevantes = resultados_relevantes[q]
        recomendados = resultados_sistema[q]
        ap = average_precision(relevantes, recomendados)
        map += ap

    if num_consultas > 0:
        map /= num_consultas

    return map


0.5882758350175267

In [ ]:

# Aplicar o MAP nas consultas
mean_average_precision(resultados_relevantes, resultados_sistema)